In [10]:
import pandas as pd
import numpy as np
import seaborn as sns
import ipywidgets as widgets
from IPython.display import display

def calculate_percent_rank_inc(data, start_date=None, end_date=None):
    if isinstance(data, pd.DataFrame):
        if data.shape[1] != 1:
            raise ValueError("DataFrame input should have exactly one column.")
        series = data.iloc[:, 0]
    elif isinstance(data, dict):
        flat_data = {pd.to_datetime(date): value for date, nested_dict in data.items() for key, value in nested_dict.items()}
        series = pd.Series(flat_data)
    elif isinstance(data, pd.Series):
        series = data
    else:
        raise TypeError("Input data should be a pandas Series, DataFrame, or dictionary.")
    
    if not isinstance(series.index, pd.DatetimeIndex):
        series.index = pd.to_datetime(series.index)
    
    if start_date or end_date:
        start_date = pd.to_datetime(start_date) if start_date else series.index.min()
        end_date = pd.to_datetime(end_date) if end_date else series.index.max()
        series = series.loc[start_date:end_date]
    
    if series.empty:
        return pd.Series([], index=series.index)
    
    ranks = series.rank(method='min').apply(lambda x: (x - 1) / (len(series) - 1))
    return ranks

def calculate_percentile_rank_surface(nested_dict, udl, date, moneyness_levels, start_date=None, end_date=None):
    try:
        date_str = pd.to_datetime(date).strftime('%Y-%m-%d')
        percentile_rank_surface = pd.DataFrame(index=moneyness_levels, columns=nested_dict[date_str][udl]['IV'].keys())

        for matu in percentile_rank_surface.columns:
            for mon in moneyness_levels:
                try:
                    values = {
                        pd.to_datetime(past_date): nested_dict[past_date][udl]['IV'][matu][mon]
                        for past_date in nested_dict
                        if udl in nested_dict[past_date] and 'IV' in nested_dict[past_date][udl]
                        and matu in nested_dict[past_date][udl]['IV'] and mon in nested_dict[past_date][udl]['IV'][matu]
                    }
                    if values:
                        series = pd.Series(values)
                        percentile_series = calculate_percent_rank_inc(series, start_date, end_date)
                        percentile_rank_surface.at[mon, matu] = percentile_series.get(pd.to_datetime(date_str), np.nan)
                    else:
                        percentile_rank_surface.at[mon, matu] = np.nan
                except KeyError:
                    percentile_rank_surface.at[mon, matu] = np.nan

        return percentile_rank_surface.T
    except Exception as e:
        print(f"An error occurred in calculate_percentile_rank_surface: {e}")
        return pd.DataFrame()

def create_vol_surface(nested_dict, date, udl, moneyness_levels):
    date_str = date if isinstance(date, str) else date.strftime('%Y-%m-%d')
    if date_str not in nested_dict:
        raise ValueError(f"Date {date_str} not found in data.")

    vol_surface = pd.DataFrame(index=moneyness_levels)
    for matu, moneyness_data in nested_dict[date_str][udl]['IV'].items():
        vol_surface[matu] = [moneyness_data.get(moneyness, np.nan) for moneyness in moneyness_levels]
    vol_surface = vol_surface.T
    vol_surface.columns = [f'{mon}' for mon in vol_surface.columns]
    vol_surface.index.name = 'Maturity'
    vol_surface = vol_surface.apply(lambda col: col.map(lambda x: round(x, 2) if pd.notna(x) else np.nan))
    return vol_surface

def ensure_numerical(df):
    return df.apply(pd.to_numeric, errors='coerce')

def style_df(df, caption):
    """
    Apply styling to a DataFrame for better visualization.
    
    Args:
    df (pd.DataFrame): The DataFrame to be styled.
    caption (str): The caption for the styled DataFrame.
    
    Returns:
    pd.io.formats.style.Styler: A Styler object with the applied styles.
    """
    df = ensure_numerical(df)

    # Handle cases where the DataFrame may contain all-NaN slices
    if df.isnull().all().all():
        print("The DataFrame contains only NaN values.")
        return df.style.set_caption(caption)

    cm = sns.light_palette("green", as_cmap=True)
    df_styled = df.style.background_gradient(cmap=cm, axis=None).format("{:.2f}").set_caption(caption)
    
    return df_styled


def calculate_z_scores(series):
    """
    Calculate z-scores for a series of observations.

    Parameters:
    series (pd.Series): Series of observations.

    Returns:
    pd.Series: Series with z-scores.
    """
    return (series - series.mean()) / series.std()

def plot_surface(nested_dict, udl, date, surface_type, moneyness_levels, start_date=None, end_date=None):
    try:
        date = pd.Timestamp(date)

        if surface_type == 'Level':
            vol_surface = create_vol_surface(nested_dict, date, udl, moneyness_levels)
            vol_surface = ensure_numerical(vol_surface)  # Ensure numerical format
            
            # Handle all-NaN columns and rows
            if vol_surface.isnull().all().all():
                print("The Volatility Surface contains only NaN values.")
                display(vol_surface.style.set_caption("Volatility Surface"))
            else:
                display(style_df(vol_surface, "Volatility Surface"))
        elif surface_type == 'Percentile':
            if start_date and end_date:
                start_date = pd.Timestamp(start_date)
                end_date = pd.Timestamp(end_date)
                if start_date > end_date:
                    print("Start date cannot be after end date.")
                    return
                
                percentile_surface = calculate_percentile_rank_surface(nested_dict, udl, date, moneyness_levels, start_date, end_date)
                percentile_surface = ensure_numerical(percentile_surface)  # Ensure numerical format
                title = f"Percentile Surface ({udl}) From: {start_date.strftime('%Y-%m-%d')} to: {end_date.strftime('%Y-%m-%d')}"
                styled_df = style_df(percentile_surface, title)
                display(styled_df)
            else:
                print("Please select start and end dates for Percentile surface.")
        elif surface_type == 'Z-score':
            if start_date and end_date:
                start_date = pd.Timestamp(start_date)
                end_date = pd.Timestamp(end_date)
                if start_date > end_date:
                    print("Start date cannot be after end date.")
                    return
                
                z_score_surface = pd.DataFrame(index=moneyness_levels, columns=nested_dict[date.strftime('%Y-%m-%d')][udl]['IV'].keys())
                for matu in z_score_surface.columns:
                    for mon in moneyness_levels:
                        try:
                            values = {
                                pd.to_datetime(past_date): nested_dict[past_date][udl]['IV'][matu][mon]
                                for past_date in nested_dict
                                if udl in nested_dict[past_date] and 'IV' in nested_dict[past_date][udl]
                                and matu in nested_dict[past_date][udl]['IV'] and mon in nested_dict[past_date][udl]['IV'][matu]
                            }
                            if values:
                                series = pd.Series(values)
                                z_scores = calculate_z_scores(series)
                                z_score_surface.at[mon, matu] = z_scores.get(pd.to_datetime(date.strftime('%Y-%m-%d')), np.nan)
                            else:
                                z_score_surface.at[mon, matu] = np.nan
                        except KeyError:
                            z_score_surface.at[mon, matu] = np.nan
                
                z_score_surface = z_score_surface.T
                z_score_surface.columns = [f'{mon}' for mon in z_score_surface.columns]
                title = f"Z-score Surface ({udl}) From: {start_date.strftime('%Y-%m-%d')} to: {end_date.strftime('%Y-%m-%d')}"
                styled_df = style_df(z_score_surface, title)
                display(styled_df)
            else:
                print("Please select start and end dates for Z-score surface.")
        else:
            print("Invalid surface type selected.")
    except KeyError as e:
        print(f"KeyError: {e} - Ensure the selected date range is within the data's date range.")
    except Exception as e:
        print(f"An error occurred: {e}")


# Example data
nested_dict = {
    '2024-05-20': {'JP_NKY': {'IV': {1: {80: 0, 90: 1}}}},
    '2024-05-21': {'JP_NKY': {'IV': {1: {80: 0, 90: 1}}}},
    '2024-05-22': {'JP_NKY': {'IV': {1: {80: -1, 90: 0}}}},
    '2024-05-23': {'JP_NKY': {'IV': {1: {80: 0, 90: 1}}}},
    '2024-05-24': {'JP_NKY': {'IV': {1: {80: -10, 90: 0}}}},
    '2024-05-25': {'JP_NKY': {'IV': {1: {80: 20, 90: 2}}}},
    '2024-05-26': {'JP_NKY': {'IV': {1: {80: 1, 90: 2}}}},
    '2024-05-27': {'JP_NKY': {'IV': {1: {80: 2, 90: 3}}}},
    '2024-05-28': {'JP_NKY': {'IV': {1: {80: 3, 90: 4}}}},
}

# Widget definitions
udl_list = ['JP_NKY', 'DE_DAX', 'GB_FTSE100', 'CH_SMI', 'IT_FTMIB', 'ES_IBEX', 'US_SPX', 'EU_STOXX50E', 'EU_SX7E', 'EU_SX7P',
            'EU_SXDP', 'US_KO', 'US_MCD', 'US_KOMO', 'EU_SXPP', 'EU_SOXP', 'HK_HSI']
moneyness_levels = [80, 90, 100, 110, 120]

udl_widget = widgets.Dropdown(
    options=udl_list,
    value='JP_NKY',
    description='UDL:',
    disabled=False,
)

date_widget = widgets.DatePicker(
    description='Date',
    value=pd.to_datetime('2024-05-28'),
    disabled=False
)

start_date_widget = widgets.DatePicker(
    description='Start Date',
    value=pd.to_datetime('2024-01-01'),
    disabled=False
)

end_date_widget = widgets.DatePicker(
    description='End Date',
    value=pd.to_datetime('2024-05-27'),
    disabled=False
)

surface_type_widget = widgets.Dropdown(
    options=['Level', 'Percentile', 'Z-score'],
    value='Level',
    description='Type:',
    disabled=False,
)

def toggle_date_widgets(surface_type):
    if surface_type in ['Percentile', 'Z-score']:
        start_date_widget.layout.display = 'block'
        end_date_widget.layout.display = 'block'
        date_widget.layout.display = 'none'
    else:
        start_date_widget.layout.display = 'none'
        end_date_widget.layout.display = 'none'
        date_widget.layout.display = 'block'

surface_type_widget.observe(lambda change: toggle_date_widgets(change['new']), names='value')

# Create interactive output
output = widgets.interactive_output(
    plot_surface,
    {
        'nested_dict': widgets.fixed(nested_dict), 
        'udl': udl_widget, 
        'date': date_widget,
        'surface_type': surface_type_widget,
        'moneyness_levels': widgets.fixed(moneyness_levels),
        'start_date': start_date_widget,
        'end_date': end_date_widget
    }
)

# Layout for widgets
left_box = widgets.VBox([udl_widget, surface_type_widget], layout=widgets.Layout(margin='10px'))
right_box = widgets.VBox([start_date_widget, end_date_widget, date_widget], layout=widgets.Layout(margin='10px'))
top_box = widgets.HBox([left_box, right_box], layout=widgets.Layout(justify_content='space-between', align_items='center', margin='10px'))
main_box = widgets.VBox([top_box, output])

# Set initial visibility
toggle_date_widgets(surface_type_widget.value)

# Display the widgets and output
display(main_box)
